<a href="https://colab.research.google.com/github/tk811/CatterPlots/blob/master/Copy_of_Intro_to_Python_Pandas_DataFrames_Box_Office_Data_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Python: Pandas DataFrames
## Box Office Data

Today, we will be exploring fundamental concepts of pandas data manipulation to prepare a data set for modeling. 

__Data set:__ Information from IMDB about movies

__Our Goal:__ Process and clean the data to prepare it for modeling to predict the gross profit of a movie.

You will need to add some code to complete this notebook.  Follow along with the instructor to find what code to add.  You will add that where the code says "\*\*\* ADD CODE HERE\*\*\*"

Have fun and good luck coding!

## Importing the packages that we'll need

One of the things that makes Python **great** for data science is all of the different libraries that exist so we don't have to code them from scratch. Tonight we'll be taking advantage of:
- [Numpy](https://numpy.org/) for scientific and mathematical computing
- [Pandas](https://pandas.pydata.org/) for data wrangling and analysis

In [0]:
# data analysis packages
import numpy as np
import pandas as pd

# default settings for pandas
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

## Import the data
Pandas can work with information from all kinds of data sources. Below, we'll import the data we need from a GitHub URL and read it into a Pandas Dataframe using the Pandas [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

In [0]:
# import data from github
data = pd.read_csv("https://github.com/autumntoney/predict_the_box_office/raw/master/movie_metadata.csv")

## Understand the data

In [0]:
# Check out the first lines of the data set
***ADD CODE HERE***

In [0]:
# Checking the size of our data (rows, columns)
data.shape

In [0]:
# Get a concise summary of the dataset
data.info()

In [0]:
# Understand the basic statistical details of the data set
***ADD CODE HERE***

__Missing Values:__ You can identify columns with missing values by looking at the `non-null` items in the `.info()` call, and the difference in number of data points in the `count` row of the `.describe()` function. For example, the `num_critic_for_reviews` column has a lower `count` than `duration`, so has missing values.

__Outliers:__ You can also identify if there are outliers in the data set by looking at the `.describe()` function's spread of data in comparison to the `min` and `max` values.  For example, the `duration` column's `max` value is 511, which would correspond to an 8.5 hour film. You might want to go back and check that the values for that film are valid, and correct or drop the values that aren't.  

NB: We don't have time to investigate outliers in this workshop.

## Clean the Data

Now that we understand the basics of what's in the data, we now need to clean the data before it's ready for modeling.  Things we'll cover:
- Duplicate data
- Missing data
- Manipulating data

NB: Most models can only process numerical data, so we will focus our cleaning on those columns.

In [0]:
# Rename columns to include units
data.rename(columns={'duration': 'duration_mins',
                    'budget': 'budget_usd',
                    'gross': 'gross_usd'}, inplace=True)
data.head(1)

### Removing duplicates
Since our data includes that of movies, we will want to check for duplicated `movie_title`, but since there could be movie remakes in here as well, we should also check the `title_year` using the Pandas [`duplicated`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function.

In [0]:
# Checking to see what movies are duplicates.  
# Sorting by movie title to see duplicates
data[data.duplicated(subset=['***ADD CODE HERE***'], keep=False)].sort_values('movie_title').head()

We do have duplicated movies in our data.  We need to drop those duplicates to remove them from our data set using the Pandas [`drop_duplicates`](https://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html) function.

In [0]:
# Drop all duplicate movie titles that were released in the same year
data = data.drop_duplicates(subset=['movie_title', 'title_year'], keep='first').copy()

### Missing values
Now that we've cleaned out the duplicates, let's take a look at the missing values.

In [0]:
# Show how many values are missing from each column
***ADD CODE HERE***

There are a good number of films missing the `aspect_ratio`, and using our knowledge of movies, it is unlikely that the aspect ratio is important in predicting the gross revenue of the film.  Let's drop the whole column using the Pandas [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) function.

In [0]:
# Drop the aspect ratio column,  axis=1 means drop the column
data.drop('aspect_ratio', axis=1, inplace=True)

Now, since we're wanting to eventually predict the `gross` revenue of the movie, let's take a look at the missing values of `gross`

In [0]:
# What percent of the gross values are missing?
print(len(data[data['gross_usd'].isna()])/len(data))
# Or, more simply
data['gross_usd'].***ADD CODE HERE***

In [0]:
# show what movies are missing the gross values
data[data['gross_usd'].isna()].head()

Since we eventually want to predict the gross revenue of a movie, having a null value for gross revenue will not help us train our model, so we should drop them using the Pandas [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) function.

In [0]:
# Drop all null values of gross
# You might be interested in how the movie performed
# so don't want to impute these values (skew the analysis)
data.dropna(subset=['gross_usd'], how='all', inplace=True)
# Check the size of the data after dropping null values
data.shape

Check out the highest-grossing movies by sorting the data.

In [0]:
# Sort all of the values by gross
data['gross_usd'].sort_values(ascending=False).head()

Knowing a bit about movies, it makes sense that the budget of a film might be important to predicting how much money the movie makes, so let's check out the missing budget values.

In [0]:
# Identifying the percent of budget values that are missing
***ADD CODE HERE***

Since we have between 5-10% of budget values missing, we can impute (or fill in) the missing values without creating too much bias in the data. 

Using our knowledge of films, I know that film budgets in recent years has increased, so just imputing the mean or median budget for all missing values would not make much sense.  To get around this, we can find the median gross revenue for each year using the Pandas [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function, and then impute based on those values using the [`fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) function.

In [0]:
# Drop movies where the year is missing
data.dropna(subset=['title_year'], how='all', inplace=True)
# Convert all years to integers
data['title_year'] = data['title_year'].astype(int)

In [0]:
# Calculate median budgets per year
# Impute the median budgets per year for missing budget data
data['budget_usd'] = data['budget_usd'].fillna(data.groupby('title_year')['budget_usd'].transform('median'))

In [0]:
# Re-check to see if there are still any missing budget values
data['budget_usd'].isna().sum()

In [0]:
# What is the movie with missing budget still?
data[data['budget_usd'].isna()].head()

There are no other movies from the year 1942, so we cannot impute based on year.  Other options:
- Impute based on the overall median budget
- Drop the film from the data
- Impute based on other data

Since it's only one film, and for the sake of time, we will drop the film from the data set. 

In [0]:
data.dropna(subset=['budget_usd'], axis=0, inplace=True)

Now, let's take a look at the different countries of origin in the data.  There are likely differences in the gross revenue of movies from different countries.

In [0]:
# Identify all of the unique countries
data['country'].unique()

In [0]:
# Find how many movies are in each country
counts = data['country'].value_counts()
counts

Since most movies are from the US, UK, and France let's just focus on movies from those three countries, so the other countries don't skew our predictions.

In [0]:
# Select just the countries with the 3 largest number of films
counts.nlargest(3)

In [0]:
# Select just the country names of the three largest
counts.***ADD CODE HERE***

In [0]:
# Select the data from only the top 3 countries
data = data[data['country'].isin(counts.nlargest(3).index)].copy()

Now, let's take a look at how our cleaning is going.

In [0]:
# Check again to see how many missing values we have
data.isna().sum()

We have very few missing values remaining -- if we had more time, we could address each individually, but we don't.  Let's drop all remaining rows that have missing values.

In [0]:
print(data.shape)
# Dropping all remaining rows that have null values
data.dropna(axis=0, inplace=True)
print(data.shape)

## Feature Engineering
Sometimes you might want to use your knowledge of the subject to create new features to help you make predictions

Here, I think that the lead actor might have an influence on the gross revenue of a film, so let's check those out to see if we can turn these into a numerical feature for modeling.

In [0]:
# Check out the actors with the most movies in the set
data['actor_1_name'].value_counts().head()

The actors with the most lead roles have 20+ lead roles in this data set.  Let's make a feature that includes whether or not the lead actor has starred in a lot of movies (20+).

For this, we will use a list comprehension, which is a concise way to create a list.  The basic syntax for this is:
- `[expression if conditional for item in list]`
  - where the `expression` is based on an item in the list, 
  - `if conditional` filters down the list
  - `for item in list` breaks down a list into individual items. 
 

In [0]:
# Identify all movie counts, select all star actors
lead_movie_counts = data['actor_1_name'].value_counts()
star_actors = lead_movie_counts[lead_movie_counts>=20].index
# Set `lead_star` = 1 if actor is in star_actors, otherwise 0
data['lead_star'] = [1 if x in star_actors else 0 for x in data['actor_1_name']]
data.head()


Maybe the era of the film might have something to do with the gross revenue.  The [history of film](http://www.historyoffilm.net/movie-eras/history-of-cinema/) documents the movie eras we can use to create our new feature.

In [0]:
# Identify the era of the film
era_bins = [0, 1910, 1926, 1940, 1954, 1976, 2000, 2100]
era_labels = ['pioneer', 'silent', 'talkies', 'golden_era',
             'changes', 'dawn_modern_film', 'modern_film']
data['era'] = pd.cut(data['title_year'], bins=era_bins, labels=era_labels)
data['era'].value_counts()


Perhaps the `content_rating` will have an impact on the gross revenue of the movie.  We already have this data, but our models will want the information in numerical form, not in words.  

To do this, we need to encode the ratings as numbers.  We can use the pandas [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function.

In [0]:
# Encoding ratings as dummy variables
content_ratings = ***ADD CODE HERE***(data['content_rating'])
content_ratings.head(2)

In [0]:
# Merge the encoded data back on to the original data
data = data.join(content_ratings)
data.head(3)

Now, we could do a similar type of encoding for any of the other categorical variables, but encoding all of the actor names would create too many columns.  We'll stick with the numerical data for now and select them using the Pandas [`select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html) function.

In [0]:
# Select columns by data type - number
numerical_data = data.***ADD CODE HERE***(include='number')
numerical_data.head()

Now our `numerical_data` DataFrame is ready for modeling!

# Take Home Challenge
- Go back and investigate outliers and clean or drop them
- Analyze your clean data (see Thinkful's Art of Visualization webinars for additional support)
- Continue to engineer features based on what you know about the subject
- Try out different models for predicting the gross revenue of a movie (see Thinkful's Intro to Predictive Modeling webinars for additional support)
- Go back and impute some of the missing data that we dropped